In [1]:
import pandas as pd
import time
import threading
from kafkaconsumer import Kafka_Consumer

In [2]:
stop_flag = False

In [3]:
def get_env_params():
    return pd.read_csv('./env_params/env_params.csv',index_col=0, header=None).squeeze().to_dict()

In [4]:
if __name__ == "__main__":
    
    params = get_env_params()

    topics = params['topics'].split('|')


    kafkaconsumers = []
    for iter,topic in enumerate(topics):
        #Create objects of kafka consumer
        kafkaconsumers.append(Kafka_Consumer(topic,[params['bootstrapservers']],str(iter)))
  
    threads = []
    for consumer in kafkaconsumers:
        # Create threads for each consumer function
        thread = threading.Thread(target=consumer.consume_and_upload
                               ,args=(params['adls_account_name'], params['adls_sas_token']
                                      ,int(params['kafkabatchsize']),params['adls_container']))
        threads.append(thread)
        thread.start()

    try:
        while not stop_flag:
            time.sleep(0.1)  # Main thread does something here, could be anything
    except KeyboardInterrupt:
        print("\nKeyboard interruption detected. Stopping all threads...")
        stop_flag = True  # Set the flag to stop threads

    finally:
        # Wait for all threads to complete

        for consumer in kafkaconsumers:
            consumer.consumer.close()
        
        for thread in threads:
            thread.join()
        print("All threads have been stopped.")

current_batch_size 0 Received message: [ConsumerRecord(topic='orders', partition=0, offset=704, timestamp=1726068532690, timestamp_type=0, key=None, value='{"OrderID": "46743741-9466-4472-ab5f-5241f323ca74", "OrderDate": "2024-09-11 20:48:30", "StoreID": 4, "GlobalID": "UK-4", "Country": "UK", "PaymentMethod": "Cash", "TotalAmount": 150}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=209, serialized_header_size=-1), ConsumerRecord(topic='orders', partition=0, offset=705, timestamp=1726068774709, timestamp_type=0, key=None, value='{"OrderID": "12e0c8b2-bad6-40fb-9948-8dec4f65d4d9", "OrderDate": "2024-09-11 20:51:30", "StoreID": 18, "GlobalID": "Ind-18", "Country": "India", "PaymentMethod": "Mobile Payment", "TotalAmount": 100}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=225, serialized_header_size=-1), ConsumerRecord(topic='orders', partition=0, offset=706, timestamp=1726068916704, timestamp_type=0, key=None, value='{"OrderID": 